In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import filters, segmentation, morphology, measure, transform

In [ ]:
# Cellule 2: Fonctions auxiliaires

def load_dicom_image(file_path):
    """
    Charge une image DICOM et la convertit en array numpy.
    Cette fonction est un placeholder, vous devrez utiliser une bibliothèque DICOM réelle.
    """
    # Utilisez pydicom ou une autre bibliothèque DICOM ici
    # Pour cet exemple, nous allons simplement créer une image synthétique
    return np.random.rand(256, 256)

def estimate_seed_point(image):
    """
    Estime le point de départ dans la cavité du ventricule gauche.
    """
    # Pour cet exemple, nous supposerons que le point central est un bon point de départ
    return np.array(image.shape) // 2

def measure_blood_statistics(image, seed):
    """
    Mesure la moyenne et l'écart-type du signal sanguin.
    """
    # Créez un masque circulaire autour du point de départ
    y, x = np.ogrid[-seed[0]:image.shape[0]-seed[0], -seed[1]:image.shape[1]-seed[1]]
    mask = x*x + y*y <= 30*30
    
    # Calculez la moyenne et l'écart-type dans cette région
    blood_region = image[mask]
    return np.mean(blood_region), np.std(blood_region)

def compensate_coil_sensitivity(image):
    """
    Compense la sensibilité de la bobine en normalisant l'image.
    """
    # Pour cet exemple, nous effectuerons une simple normalisation
    return (image - np.min(image)) / (np.max(image) - np.min(image))

def estimate_myocardial_intensity(image, seed):
    """
    Estime l'intensité du signal myocardique.
    """
    # Pour cet exemple, nous supposerons que le myocarde a une intensité légèrement inférieure au sang
    blood_mean, _ = measure_blood_statistics(image, seed)
    return blood_mean * 0.7

def segment_lv(image, myoc_mean, blood_std):
    """
    Segmente le ventricule gauche en utilisant un seuillage itératif.
    """
    threshold = myoc_mean + 2 * blood_std
    return image > threshold

In [ ]:
# Cellule 3: Implémentation de l'algorithme de seuillage itératif

def iterative_thresholding(image):
    # Étape 1: Estimer le point de départ initial
    seed = estimate_seed_point(image)
    
    # Étape 2: Mesurer la moyenne et l'écart-type du signal sanguin
    blood_mean, blood_std = measure_blood_statistics(image, seed)
    
    # Étape 3: Compenser la sensibilité de la bobine
    image = compensate_coil_sensitivity(image)
    
    # Étape 4: Estimer l'intensité du signal myocardique
    myoc_mean = estimate_myocardial_intensity(image, seed)
    
    # Étape 5: Segmenter le VG et mesurer le volume sanguin
    lv_mask = segment_lv(image, myoc_mean, blood_std)
    blood_volume = np.sum(lv_mask)
    
    return lv_mask, blood_volume

In [ ]:
# Cellule 4: Fonctions pour le modèle de contour actif

def generate_circular_map(image):
    """
    Génère une carte circulaire de l'image.
    """
    y, x = np.ogrid[:image.shape[0], :image.shape[1]]
    center = np.array(image.shape) // 2
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    theta = np.arctan2(y - center[0], x - center[1])
    return r, theta

def extract_filter_edges(image, initial_contour):
    """
    Extrait et filtre les informations de bord.
    """
    edges = filters.canny(image)
    # Filtrer les bords en fonction du contour initial
    mask = np.zeros_like(edges, dtype=bool)
    mask[tuple(initial_contour.T)] = True
    mask = morphology.dilation(mask, morphology.disk(5))
    return edges * mask

def define_external_force(edges):
    """
    Définit la force externe pour le modèle de contour actif.
    """
    return filters.gaussian(edges, sigma=2)

def define_constraint(image, myoc_mean, blood_std):
    """
    Définit la contrainte pour le modèle de contour actif.
    """
    return (image > myoc_mean + 2*blood_std) | (image < myoc_mean * 0.4)

In [ ]:
# Cellule 5: Implémentation du modèle de contour actif

def active_contour_model(image, initial_contour, myoc_mean, blood_std):
    # Générer la carte circulaire
    r, theta = generate_circular_map(image)
    
    # Extraire et filtrer les informations de bord
    edges = extract_filter_edges(image, initial_contour)
    
    # Définir la force externe et la contrainte
    external_force = define_external_force(edges)
    constraint = define_constraint(image, myoc_mean, blood_std)
    
    # Appliquer le modèle de contour actif
    contour = segmentation.active_contour(
        external_force,
        initial_contour,
        alpha=0.015,
        beta=10,
        gamma=0.001
    )
    
    # Mettre à jour le contour épicardique et transformer les coordonnées
    contour = transform.resize(contour, (len(contour), 2))
    
    return contour

In [ ]:
# Cellule 6: Fonction principale ITHACA

def ithaca_segmentation(image):
    # Segmentation endocardique
    endo_mask, blood_volume = iterative_thresholding(image)
    
    # Trouver le contour initial pour la segmentation épicardique
    initial_contour = measure.find_contours(endo_mask, 0.5)[0]
    
    # Estimer les statistiques du myocarde pour le modèle de contour actif
    myoc_mean = estimate_myocardial_intensity(image, estimate_seed_point(image))
    _, blood_std = measure_blood_statistics(image, estimate_seed_point(image))
    
    # Segmentation épicardique
    epi_contour = active_contour_model(image, initial_contour, myoc_mean, blood_std)
    
    # Calculer la masse myocardique
    epi_mask = measure.grid_points_in_poly(image.shape, epi_contour)
    myoc_mask = epi_mask ^ endo_mask  # XOR pour obtenir uniquement le myocarde
    myoc_mass = np.sum(myoc_mask) * 1.05  # densité de 1.05 g/mL
    
    return endo_mask, epi_contour, blood_volume, myoc_mass

In [ ]:
# Cellule 7: Test de l'algorithme

# Charger une image de test
image = load_dicom_image('path_to_your_dicom_file.dcm')

# Appliquer l'algorithme ITHACA
endo_mask, epi_contour, blood_volume, myoc_mass = ithaca_segmentation(image)

# Afficher les résultats
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.imshow(image, cmap='gray')
ax1.set_title('Image originale')

ax2.imshow(image, cmap='gray')
ax2.contour(endo_mask, colors='r', levels=[0.5])
ax2.plot(epi_contour[:, 1], epi_contour[:, 0], 'b-')
ax2.set_title('Segmentation ITHACA')

plt.tight_layout()
plt.show()

print(f"Volume sanguin: {blood_volume:.2f} mL")
print(f"Masse myocardique: {myoc_mass:.2f} g")